# Step-by-step run of alphazero self-play & training.


In [1]:
import os
import time
from pathlib import Path
from collections import defaultdict, Counter
import asyncio
from typing import Callable

import numpy as np
import torch
import torch.nn.functional as F

# Game and players
from rgi.rgizero.games.connect4 import Connect4Game
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.alphazero import play_game

from notebook_utils import reload_local_modules

print("✅ Imports successful")

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
    device = 'cpu'
print(f'Using device: {device}')
assert device in ('cuda', 'mps'), f"No accelerator available, device={device}"

# Allow asyncio to work with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Increase numpy print width
np.set_printoptions(linewidth=300)

%load_ext line_profiler

✅ Imports successful
Using device: mps


In [2]:
DEBUG_MODE = True     # Set options to make debugger work properly. Single worker, etc.
LOAD_MODEL = False
TRAIN_MODEL = True
MODEL_SIZE = "tiny"  # "tiny" or "small" or "large" or "xl"
NUM_SIMULATIONS = 200
RUN_GENERATIONS = True

# If False, we still load previous games from disk.
NUM_GAMES = 10_000
MAX_TRAINING_EPOCHS = 10
TRAIN_BATCH_SIZE = 2048
MAX_TRAINING_ITERS = 1_000_000 // TRAIN_BATCH_SIZE
CONFIG_ALIAS = f'trajectory_sims-{NUM_SIMULATIONS}_games-{NUM_GAMES}_size-{MODEL_SIZE}_train-{MAX_TRAINING_ITERS}_x1'
NUM_GENERATIONS = 5

# DEBUG: Update batch_size after config_alias
MODEL_SIZE = "small"
MAX_TRAINING_ITERS = 100_000_000 // TRAIN_BATCH_SIZE
MAX_TRAINING_EPOCHS = 10_000


## Step 1: Set up history-wrapped game


In [3]:
from rgi.rgizero.games.history_wrapper import HistoryTrackingGame
from rgi.rgizero.data.trajectory_dataset import Vocab
from rgi.rgizero.common import TOKENS

base_game, max_game_length = Connect4Game(connect_length=4), 7*6

game = HistoryTrackingGame(base_game)
state_0 = game.initial_state()
block_size = max_game_length + 2
all_actions = game.all_actions()
action_vocab = Vocab(itos=[TOKENS.START_OF_GAME] + list(all_actions))
n_max_context = max_game_length + 2
game_name = base_game.__class__.__name__

print("✅ Using HistoryTrackingGame from module")
print(f"Game: {game_name}, Players: {game.num_players(state_0)}, Actions: {list(game.all_actions())}")

DATA_DIR = Path.cwd().parent / "data" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating data dir: ", DATA_DIR)
os.makedirs(DATA_DIR, exist_ok=True)

MODEL_DIR = Path.cwd().parent / "models" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating model dir: ", MODEL_DIR)
os.makedirs(MODEL_DIR, exist_ok=True)


✅ Using HistoryTrackingGame from module
Game: Connect4Game, Players: 2, Actions: [1, 2, 3, 4, 5, 6, 7]
Creating data dir:  /Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1
Creating model dir:  /Users/rodo/src/rgi3/models/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1


## Step 2: Create random generation_0 model


In [4]:
def print_dataset_stats(dataset_path, split_name):
    """Print statistics about a loaded trajectory dataset."""
    td = TrajectoryDataset(dataset_path.parent, split_name, block_size=n_max_context)
    
    # Calculate basic stats
    num_trajectories = len(td)
    total_actions = td._num_actions
    avg_trajectory_length = total_actions / num_trajectories if num_trajectories > 0 else 0
    
    # Get trajectory lengths, winners, and first moves
    trajectory_lengths = []
    winners = []
    first_moves = []
    
    for i in range(num_trajectories):
        start_idx = td.boundaries[i]
        end_idx = td.boundaries[i + 1]
        traj_length = end_idx - start_idx
        trajectory_lengths.append(traj_length)
        
        # Get winner from final values (values are the same throughout trajectory)
        # Values are in range [-1, 1] where positive means player 1 advantage
        final_values = td.value_data[start_idx]  # shape: (num_players,)
        if final_values[0] > final_values[1]:
            winners.append(1)
        elif final_values[1] > final_values[0]:
            winners.append(2)
        else:
            winners.append(None)  # Draw
        
        # Get first move (decode from vocab)
        first_action_encoded = td.action_data[start_idx]
        first_action = action_vocab.decode([first_action_encoded])[0]
        first_moves.append(first_action)
    
    # Print basic stats
    print(f"Dataset Stats:")
    print(f"  Trajectories: {num_trajectories}")
    print(f"  Total actions: {total_actions}")
    print(f"  Avg trajectory length: {avg_trajectory_length:.2f}")
    print(f"  Trajectory length - min: {min(trajectory_lengths)}, max: {max(trajectory_lengths)}, mean: {np.mean(trajectory_lengths):.2f}")
    
    # Print winner stats (similar to print_game_stats)
    print(f"Winner Stats:")
    winner_stats = Counter(winners)
    total_games = num_trajectories
    win1_pct = 100 * winner_stats[1] / total_games if total_games > 0 else 0
    win2_pct = 100 * winner_stats[2] / total_games if total_games > 0 else 0
    print(f"  Winner counts: win[1]={win1_pct:.2f}% win[2]={win2_pct:.2f}%, n={total_games}")
    
    # Print stats by initial move
    print(f"Winner Stats by initial move:")
    move_stats = defaultdict(Counter)
    for first_move, winner in zip(first_moves, winners):
        move_stats[first_move][winner] += 1
    
    for action in sorted(move_stats.keys()):
        counts = move_stats[action]
        total = sum(counts.values())
        win1_pct = 100 * counts[1] / total if total > 0 else 0
        win2_pct = 100 * counts[2] / total if total > 0 else 0
        draw_pct = 100 * counts[None] / total if total > 0 else 0
        print(f"  a={action}: n={total:3} win[1]={win1_pct:.2f}% counts={counts}, win[2]={win2_pct:.2f}% draw={draw_pct:.2f}%")

def print_model_stats(model, config_alias=""):
    """Print statistics about a model."""
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())
    num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model Stats:")
    print(f"  Config: {model.config}")
    print(f"  Total parameters: {num_params:,}")
    print(f"  Trainable parameters: {num_trainable:,}")
    if config_alias:
        print(f"  Config alias: {config_alias}")


In [5]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer
from rgi.rgizero.models.transformer import TransformerConfig

model_config_dict = {
    "tiny": TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=8),
    "small": TransformerConfig(n_max_context=n_max_context, n_layer=4, n_head=4, n_embd=32),
    "large": TransformerConfig(n_max_context=n_max_context, n_layer=8, n_head=8, n_embd=128),
    "xl": TransformerConfig(n_max_context=n_max_context, n_layer=16, n_head=16, n_embd=256),
}


def create_random_model(config: TransformerConfig, action_vocab_size, num_players,  seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed) # Ensure numpy operations are also seeded
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    model = ActionHistoryTransformer(config=config, action_vocab_size=action_vocab_size, num_players=num_players)
    model.to(device)
    return model

# Make model initialization deterministic
model_config = model_config_dict[MODEL_SIZE]
# TODO: Use MODEL_SIZE!
# model_config = model_config_dict["small"] # Override to see if we can fit better.
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)

# Step 3: Define play & generation code


In [6]:
from rgi.rgizero.models.action_history_transformer import AsyncNetworkEvaluator, ActionHistoryTransformerEvaluator
from rgi.rgizero.players.alphazero import play_game_async
from tqdm.asyncio import tqdm

async def play_games_async(num_games: int, player_factory: Callable[[], AlphazeroPlayer], max_concurrent_games: int = 1000):
    sem = asyncio.Semaphore(max_concurrent_games)
    tasks = []
    async def create_player_and_create_game():
        async with sem:
            t0 = time.time()
            player = player_factory()
            game_result = await play_game_async(game, [player, player])
            t1 = time.time()
            game_result['time'] = t1 - t0
            return game_result

    tasks = [create_player_and_create_game() for _ in range(num_games)]
    results = await tqdm.gather(*tasks)   # same as asyncio.gather, but with a progress bar
    return results

async def play_generation_async(model, num_games, simulations=NUM_SIMULATIONS, max_concurrent_games=1024):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    async_evaluator = AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=max_concurrent_games, verbose=False)

    master_rng = np.random.default_rng(42)
    async_player_factory = lambda: AlphazeroPlayer(game, async_evaluator, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=simulations)

    await async_evaluator.start()
    results = await play_games_async(num_games=num_games, player_factory=async_player_factory, max_concurrent_games=max_concurrent_games)
    await async_evaluator.stop()
    return results

def print_game_stats(results):
    print("Winner Stats:")
    winner_stats = Counter(result['winner'] for result in results)
    print(f"Winner counts: win[1]={100*winner_stats[1]/sum(winner_stats.values()):.2f}% win[2]={100*winner_stats[2]/sum(winner_stats.values()):.2f}%, n={sum(winner_stats.values())}, draw={100*winner_stats[None]/sum(winner_stats.values()):.2f}%")
    game_lengths = [len(result['action_history']) for result in results]
    print(f"Game Length min: {min(game_lengths)}, max: {max(game_lengths)}, mean: {np.mean(game_lengths):.2f}")
    print("Winner Stats by initial move:")
    dd = defaultdict(Counter)
    for result in results:
        dd[result['action_history'][0]][result['winner']] += 1
    for action, counts in sorted(dd.items()):
        print(f"  a={action}: n={sum(counts.values()):3} win[1]={100*counts[1]/sum(counts.values()):.2f}% counts={counts}, win[2]={100*counts[2]/sum(counts.values()):.2f}% draw={100*counts[None]/sum(counts.values()):.2f}%")

## Step 3: Confirm we can read & write to trajectory_dataset


In [7]:
from rgi.rgizero.data.trajectory_dataset import TrajectoryDatasetBuilder, TrajectoryDataset, build_trajectory_loader
reload_local_modules(verbose=False)

def add_trajectory(game_result, vocab, td_builder):
    action_history = game_result['action_history']
    trajectory_length = len(action_history)
    legal_policies = game_result['legal_policies']
    legal_action_idx = game_result['legal_action_idx']
    rewards = game_result['rewards']

    # Translation key for converting legal_action_ids to vocab_action_idx.
    action_idx_to_vocab_idx = vocab.encode(all_actions)

    fixed_width_policies = np.zeros((trajectory_length, vocab.vocab_size))
    for i in range(trajectory_length):
        vocab_action_idx = action_idx_to_vocab_idx[legal_action_idx[i]]
        fixed_width_policies[i, vocab_action_idx] = legal_policies[i]

    encoded_action_history = vocab.encode(action_history)
    tiled_rewards = np.tile(rewards, (trajectory_length, 1))  # shape (num_players,) -> (num_moves, num_players)
    
    td_builder.add_trajectory(actions=encoded_action_history, fixed_width_policies=fixed_width_policies, values=tiled_rewards)

def write_trajectory_dataset(results, action_vocab, generation_id):
    td_builder = TrajectoryDatasetBuilder(action_vocab)
    for game_result in results:
        add_trajectory(game_result, action_vocab, td_builder)

    trajectory_path = td_builder.save(DATA_DIR, f"gen-{generation_id}")
    return trajectory_path

## Train model


In [8]:
from rgi.rgizero.train import Trainer, TrainConfig

LEARNING_RATE = 0.05

train_config = TrainConfig(
    model_name="connect4-e2e",
    model_version="v1",

    eval_interval = 1000,  # keep frequent because we'll overfit
    eval_iters = 20,
    log_interval = 100,  # don't print too too often
    max_epochs = MAX_TRAINING_EPOCHS,

    # we expect to overfit on this small dataset, so only save when val improves
    always_save_checkpoint = False,

    gradient_accumulation_steps = 1,
    batch_size = TRAIN_BATCH_SIZE,

    learning_rate = LEARNING_RATE,  # with baby networks can afford to go a bit higher
    max_iters = MAX_TRAINING_ITERS,
    lr_decay_iters = MAX_TRAINING_ITERS,  # make equal to max_iters usually
    min_lr = LEARNING_RATE / 10,  # learning_rate / 10 usually
    beta2 = 0.99,  # make a bit bigger because number of tokens per iter is small

    warmup_iters = 0,  # not super necessary potentially
)

def train_model(model, training_splits, train_config):
    # Load dataset
    num_workers = 0 if DEBUG_MODE else 4

    trajectory_loader = build_trajectory_loader(
        DATA_DIR, training_splits, block_size=n_max_context, batch_size=train_config.batch_size,
        device=device, workers=num_workers, shuffle=True)
        
    trainer = Trainer(
        model=model,
        train_config=train_config,
        train_loader=trajectory_loader,
        val_loader=trajectory_loader,  # TODO: Create separate validation loader
        device=device
    )

    trainer.train()
    return model, trainer

In [9]:
import dataclasses

def get_model_path(generation_id):
    return MODEL_DIR / f"gen-{generation_id}.pt"

def save_model(model, trainer, generation_id):
    # Save model
    model_path = get_model_path(generation_id)

    checkpoint = {
        'model': model.state_dict(),
        'model_config': dataclasses.asdict(model.config),
        'vocab': action_vocab.to_dict(),
        'iter_num': trainer.iter_num,
        'best_val_loss': trainer.best_val_loss,
        'num_players': game.num_players(state_0),
    }
    torch.save(checkpoint, model_path)
    return model_path

def load_model(generation_id):
    model_path = get_model_path(generation_id)
    loaded_checkpoint = torch.load(model_path)
    loaded_model = ActionHistoryTransformer(
        config=TransformerConfig(**loaded_checkpoint['model_config']),
        action_vocab_size=loaded_checkpoint['vocab']['vocab_size'],
        num_players=loaded_checkpoint['num_players']
    )
    loaded_model.load_state_dict(loaded_checkpoint['model']) 
    loaded_model.to(device)
    return loaded_model


In [10]:
# Do a single generation of play & train
async def run_generation(model, num_games, num_simulations, generation_id):
    print(f"\n\n## Running generation {generation_id} for config_alias={CONFIG_ALIAS}")
    split_name = f"gen-{generation_id}"
    expected_trajectory_path = DATA_DIR / split_name
    if not expected_trajectory_path.exists():
        print(f"Playing {num_games} games, simulations={num_simulations}, model_size={MODEL_SIZE}")
        results = await play_generation_async(model, num_games=NUM_GAMES, simulations=NUM_SIMULATIONS)
        print_game_stats(results)
        trajectory_path = write_trajectory_dataset(results, action_vocab, generation_id)
        assert trajectory_path == expected_trajectory_path
    else:
        print(f"Loading trajectory from {expected_trajectory_path}")
        print_dataset_stats(expected_trajectory_path, split_name)
        trajectory_path = expected_trajectory_path
        results = None

    model_path = get_model_path(generation_id)
    if not model_path.exists():
        print(f"Training model on {split_name}")
        training_splits = [f"gen-{i}" for i in range(1, generation_id+1)]
        # TODO: We're continuing training on a previosu model here ... should we train a new model from scratch?
        print(train_config)
        updated_model, trainer = train_model(model, training_splits, train_config)
        save_model(updated_model, trainer, generation_id)
    else:
        print(f"Loading model from {model_path}")
        updated_model = load_model(generation_id)
        print_model_stats(updated_model, config_alias=MODEL_SIZE)

    return results, trajectory_path, updated_model

In [11]:
results_dict = {}
trajectory_paths_dict = {}
model_dict = {0: model_0}

current_model = model_dict[0]
if RUN_GENERATIONS:
    for generation_id in range(1, NUM_GENERATIONS+1):
        current_model = model_dict[generation_id-1]
        results_i, trajectory_path_i, model_i = await run_generation(current_model, num_games=NUM_GAMES, num_simulations=NUM_SIMULATIONS, generation_id=generation_id)
        results_dict[generation_id] = results_i
        trajectory_paths_dict[generation_id] = trajectory_path_i
        model_dict[generation_id] = model_i

## refactor, learning_rate = 0.05, warmup_iters=0
# step 0: train loss 2.7801, val loss 2.7801
# iter 0/5/488: loss 2.7801, time 611.02ms
# iter 100/105/488: loss 2.5840, time 63.73ms
# iter 200/205/488: loss 2.5958, time 62.98ms
# iter 300/305/488: loss 2.5835, time 60.15ms
# iter 400/405/488: loss 2.5793, time 63.62ms

# ## model = small
# step 0: train loss 2.7741, val loss 2.7741
# iter 0/5/488: loss 2.7743, time 1624.89ms
# iter 100/105/488: loss 2.6157, time 141.39ms
# iter 200/205/488: loss 2.6120, time 161.22ms
# iter 300/305/488: loss 2.5983, time 203.82ms


# using fused AdamW: False
# step 0: train loss 2.7801, val loss 2.7801
# iter 0/5/4882: loss 2.7801, time 1422.53ms
# iter 100/105/4882: loss 2.5970, time 110.71ms
# iter 200/205/4882: loss 2.5962, time 116.96ms
# iter 300/305/4882: loss 2.5917, time 160.95ms
# iter 400/405/4882: loss 2.5885, time 63.37ms
# iter 500/505/4882: loss 2.5912, time 65.25ms
# iter 600/605/4882: loss 2.6000, time 67.49ms
# iter 700/705/4882: loss 2.5780, time 61.43ms
# iter 800/805/4882: loss 2.5864, time 265.56ms
# iter 900/905/4882: loss 2.5857, time 263.09ms
# step 1000: train loss 2.5849, val loss 2.5847
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 1000/1005/4882: loss 2.5844, time 1812.58ms
# iter 1100/1105/4882: loss 2.5832, time 62.89ms
# iter 1200/1205/4882: loss 2.5743, time 95.25ms
# iter 1300/1305/4882: loss 2.5720, time 324.18ms
# iter 1400/1405/4882: loss 2.5880, time 73.66ms
# iter 1500/1505/4882: loss 2.5745, time 295.39ms
# iter 1600/1605/4882: loss 2.5726, time 76.05ms
# iter 1700/1705/4882: loss 2.5670, time 63.20ms
# iter 1800/1805/4882: loss 2.5720, time 62.66ms
# iter 1900/1905/4882: loss 2.5694, time 449.06ms
# step 2000: train loss 2.5806, val loss 2.5806
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 2000/2005/4882: loss 2.5893, time 920.12ms
# iter 2100/2105/4882: loss 2.5686, time 430.53ms
# iter 2200/2205/4882: loss 2.5741, time 63.05ms
# iter 2300/2305/4882: loss 2.5679, time 60.90ms
# iter 2400/2405/4882: loss 2.5754, time 69.07ms
# iter 2500/2505/4882: loss 2.5673, time 68.33ms
# iter 2600/2605/4882: loss 2.5648, time 66.26ms
# iter 2700/2705/4882: loss 2.5622, time 69.76ms
# iter 2800/2805/4882: loss 2.5541, time 143.65ms
# iter 2900/2905/4882: loss 2.5634, time 66.40ms
# step 3000: train loss 2.5550, val loss 2.5547
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 3000/3005/4882: loss 2.5545, time 975.15ms
# iter 3100/3105/4882: loss 2.5594, time 63.55ms
# iter 3200/3205/4882: loss 2.5499, time 64.17ms
# iter 3300/3305/4882: loss 2.5481, time 70.28ms
# iter 3400/3405/4882: loss 2.5565, time 73.58ms
# iter 3500/3505/4882: loss 2.5602, time 72.22ms
# iter 3600/3605/4882: loss 2.5429, time 88.68ms
# iter 3700/3705/4882: loss 2.5259, time 63.15ms
# iter 3800/3805/4882: loss 2.5346, time 66.07ms
# iter 3900/3905/4882: loss 2.5386, time 73.50ms
# step 4000: train loss 2.5350, val loss 2.5345
# saving checkpoint to /Users/rodo/src/rgi3/models/connect4-e2e/v1
# iter 4000/4005/4882: loss 2.5424, time 1217.41ms
# iter 4100/4105/4882: loss 2.5290, time 101.01ms
# iter 4200/4205/4882: loss 2.5323, time 61.94ms
# iter 4300/4305/4882: loss 2.5250, time 72.57ms
# iter 4400/4405/4882: loss 2.5243, time 68.38ms
# iter 4500/4505/4882: loss 2.5331, time 73.33ms
# iter 4600/4605/4882: loss 2.5246, time 101.00ms
# iter 4700/4705/4882: loss 2.5336, time 67.27ms
# iter 4800/4805/4882: loss 2.5170, time 79.40ms



## Running generation 1 for config_alias=trajectory_sims-200_games-10000_size-tiny_train-488_x1
Loading trajectory from /Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-200_games-10000_size-tiny_train-488_x1/gen-1
Dataset Stats:
  Trajectories: 10000
  Total actions: 140493
  Avg trajectory length: 14.05
  Trajectory length - min: 7, max: 42, mean: 14.05
Winner Stats:
  Winner counts: win[1]=61.93% win[2]=38.06%, n=10000
Winner Stats by initial move:
  a=1: n=1564 win[1]=51.92% counts=Counter({1: 812, 2: 752}), win[2]=48.08% draw=0.00%
  a=2: n=1344 win[1]=57.37% counts=Counter({1: 771, 2: 573}), win[2]=42.63% draw=0.00%
  a=3: n=1297 win[1]=65.23% counts=Counter({1: 846, 2: 451}), win[2]=34.77% draw=0.00%
  a=4: n=1493 win[1]=76.36% counts=Counter({1: 1140, 2: 353}), win[2]=23.64% draw=0.00%
  a=5: n=1525 win[1]=66.49% counts=Counter({1: 1014, 2: 511}), win[2]=33.51% draw=0.00%
  a=6: n=1401 win[1]=62.03% counts=Counter({1: 869, 2: 532}), win[2]=37.97% draw=0.00%
 

100%|██████████| 10000/10000 [18:54<00:00,  8.82it/s]


Winner Stats:
Winner counts: win[1]=46.45% win[2]=53.55%, n=10000, draw=0.00%
Game Length min: 7, max: 38, mean: 13.62
Winner Stats by initial move:
  a=1: n=933 win[1]=34.19% counts=Counter({2: 614, 1: 319}), win[2]=65.81% draw=0.00%
  a=2: n=328 win[1]=20.12% counts=Counter({2: 262, 1: 66}), win[2]=79.88% draw=0.00%
  a=3: n=1647 win[1]=43.90% counts=Counter({2: 924, 1: 723}), win[2]=56.10% draw=0.00%
  a=4: n=4260 win[1]=57.54% counts=Counter({1: 2451, 2: 1809}), win[2]=42.46% draw=0.00%
  a=5: n=1508 win[1]=41.71% counts=Counter({2: 879, 1: 629}), win[2]=58.29% draw=0.00%
  a=6: n=330 win[1]=26.06% counts=Counter({2: 244, 1: 86}), win[2]=73.94% draw=0.00%
  a=7: n=994 win[1]=37.32% counts=Counter({2: 623, 1: 371}), win[2]=62.68% draw=0.00%
Training model on gen-2
TrainConfig(model_name='connect4-e2e', model_version='v1', eval_interval=1000, log_interval=100, eval_iters=20, eval_only=False, always_save_checkpoint=False, wandb_log=False, gradient_accumulation_steps=1, batch_size=2048

100%|██████████| 10000/10000 [15:32<00:00, 10.72it/s]


Winner Stats:
Winner counts: win[1]=52.01% win[2]=47.96%, n=10000, draw=0.03%
Game Length min: 7, max: 42, mean: 14.15
Winner Stats by initial move:
  a=1: n= 94 win[1]=43.62% counts=Counter({2: 53, 1: 41}), win[2]=56.38% draw=0.00%
  a=2: n=109 win[1]=32.11% counts=Counter({2: 74, 1: 35}), win[2]=67.89% draw=0.00%
  a=3: n=103 win[1]=22.33% counts=Counter({2: 80, 1: 23}), win[2]=77.67% draw=0.00%
  a=4: n=9218 win[1]=53.38% counts=Counter({1: 4921, 2: 4295, None: 2}), win[2]=46.59% draw=0.02%
  a=5: n=132 win[1]=50.00% counts=Counter({1: 66, 2: 65, None: 1}), win[2]=49.24% draw=0.76%
  a=6: n=148 win[1]=34.46% counts=Counter({2: 97, 1: 51}), win[2]=65.54% draw=0.00%
  a=7: n=196 win[1]=32.65% counts=Counter({2: 132, 1: 64}), win[2]=67.35% draw=0.00%
Training model on gen-3
TrainConfig(model_name='connect4-e2e', model_version='v1', eval_interval=1000, log_interval=100, eval_iters=20, eval_only=False, always_save_checkpoint=False, wandb_log=False, gradient_accumulation_steps=1, batch_si

100%|██████████| 10000/10000 [16:21<00:00, 10.19it/s]


Winner Stats:
Winner counts: win[1]=45.70% win[2]=54.30%, n=10000, draw=0.00%
Game Length min: 7, max: 39, mean: 13.94
Winner Stats by initial move:
  a=1: n=139 win[1]=38.85% counts=Counter({2: 85, 1: 54}), win[2]=61.15% draw=0.00%
  a=2: n=7686 win[1]=43.98% counts=Counter({2: 4306, 1: 3380}), win[2]=56.02% draw=0.00%
  a=3: n=349 win[1]=42.69% counts=Counter({2: 200, 1: 149}), win[2]=57.31% draw=0.00%
  a=4: n=882 win[1]=64.06% counts=Counter({1: 565, 2: 317}), win[2]=35.94% draw=0.00%
  a=5: n=319 win[1]=57.68% counts=Counter({1: 184, 2: 135}), win[2]=42.32% draw=0.00%
  a=6: n= 94 win[1]=39.36% counts=Counter({2: 57, 1: 37}), win[2]=60.64% draw=0.00%
  a=7: n=531 win[1]=37.85% counts=Counter({2: 330, 1: 201}), win[2]=62.15% draw=0.00%
Training model on gen-4
TrainConfig(model_name='connect4-e2e', model_version='v1', eval_interval=1000, log_interval=100, eval_iters=20, eval_only=False, always_save_checkpoint=False, wandb_log=False, gradient_accumulation_steps=1, batch_size=2048, le

100%|██████████| 10000/10000 [16:39<00:00, 10.00it/s]


Winner Stats:
Winner counts: win[1]=58.39% win[2]=41.61%, n=10000, draw=0.00%
Game Length min: 7, max: 42, mean: 13.73
Winner Stats by initial move:
  a=1: n=462 win[1]=37.66% counts=Counter({2: 288, 1: 174}), win[2]=62.34% draw=0.00%
  a=2: n=1005 win[1]=54.63% counts=Counter({1: 549, 2: 456}), win[2]=45.37% draw=0.00%
  a=3: n=1499 win[1]=48.10% counts=Counter({2: 778, 1: 721}), win[2]=51.90% draw=0.00%
  a=4: n=5503 win[1]=67.85% counts=Counter({1: 3734, 2: 1769}), win[2]=32.15% draw=0.00%
  a=5: n=864 win[1]=41.32% counts=Counter({2: 507, 1: 357}), win[2]=58.68% draw=0.00%
  a=6: n=191 win[1]=65.45% counts=Counter({1: 125, 2: 66}), win[2]=34.55% draw=0.00%
  a=7: n=476 win[1]=37.61% counts=Counter({2: 297, 1: 179}), win[2]=62.39% draw=0.00%
Training model on gen-5
TrainConfig(model_name='connect4-e2e', model_version='v1', eval_interval=1000, log_interval=100, eval_iters=20, eval_only=False, always_save_checkpoint=False, wandb_log=False, gradient_accumulation_steps=1, batch_size=204

# Sanity check models


In [12]:
# Play single game
result = await play_generation_async(current_model, num_games=1, simulations=NUM_SIMULATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [13]:
# Inspect training data
td_array = [TrajectoryDataset(DATA_DIR, f"gen-{generation_id}", block_size=n_max_context) for generation_id in range(1, NUM_GENERATIONS+1)]

In [14]:
# [td for td in td_array]
unrolled = [(generation+1, d) for generation, td in enumerate(td_array) for d in td]

# gen, d = unrolled[0], 
# d.action[:2]
# d.value[0]

dd = defaultdict(lambda: defaultdict(lambda: torch.tensor([0., 0.])))

for gen, d in unrolled:
    for g in ['*', gen]:    
        dd[tuple(tuple(d.action[:0].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:1].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:2].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:3].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:4].tolist()))][g] += d.value[0]

print(f"len(dd) = {len(dd)}")


len(dd) = 401


In [15]:
def eval_prefix(model, game, prefix):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    state = game.initial_state()
    for action in prefix:
        state = game.next_state(state, action)
    legal_actions = game.legal_actions(state)
    result = serial_evaluator.evaluate(game, state, legal_actions)
    return result


In [16]:
## Someting is borked? Player1 win percent should be much higher??
def compare_model_vs_data(model, game, dd):
    prefix_list = [
        (0,), 
        (0,1), (0,2), (0,3), (0,4), (0,5), (0,6), (0,7),
        (0,1,1), (0,1,2), (0,1,3), (0,1,4), (0,1,5), (0,1,6), (0,1,7),
        (0,4,1), (0,4,2), (0,4,3), (0,4,4), (0,4,5), (0,4,6), (0,4,7)]
    for prefix in prefix_list:
        print(f"\nprefix={prefix}")
        for gen, counts in dd[prefix].items():
            print(f"gen={gen}: {counts}, win_pct={100*counts[0]/sum(counts):.2f}%, sum={sum(counts)}")
        assert prefix[0] == 0
        actions = prefix[1:]
        eval_result = eval_prefix(model, game, actions)
        # print(f'legal_policy={eval_result.legal_policy}')
        # print(f'player_values={eval_result.player_values}')
        print(f'player_probs={(eval_result.player_values+1)/2}')

compare_model_vs_data(current_model, game, dd)



prefix=(0,)
gen=*: tensor([26450., 23550.]), win_pct=52.90%, sum=50000.0
gen=1: tensor([6193.5000, 3806.5000]), win_pct=61.94%, sum=10000.0
gen=2: tensor([4645., 5355.]), win_pct=46.45%, sum=10000.0
gen=3: tensor([5202.5000, 4797.5000]), win_pct=52.03%, sum=10000.0
gen=4: tensor([4570., 5430.]), win_pct=45.70%, sum=10000.0
gen=5: tensor([5839., 4161.]), win_pct=58.39%, sum=10000.0
player_probs=[0.6513413 0.3486587]

prefix=(0, 1)
gen=*: tensor([1400., 1792.]), win_pct=43.86%, sum=3192.0
gen=1: tensor([812., 752.]), win_pct=51.92%, sum=1564.0
gen=2: tensor([319., 614.]), win_pct=34.19%, sum=933.0
gen=3: tensor([41., 53.]), win_pct=43.62%, sum=94.0
gen=4: tensor([54., 85.]), win_pct=38.85%, sum=139.0
gen=5: tensor([174., 288.]), win_pct=37.66%, sum=462.0
player_probs=[0.5495784 0.4504216]

prefix=(0, 2)
gen=*: tensor([4801., 5671.]), win_pct=45.85%, sum=10472.0
gen=1: tensor([771., 573.]), win_pct=57.37%, sum=1344.0
gen=2: tensor([ 66., 262.]), win_pct=20.12%, sum=328.0
gen=3: tensor([3

In [17]:
# Copy model
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)
if RUN_GENERATIONS:
    model_1 = load_model(1)


In [18]:
print("\n\n### Model 0")
print(model_0.action_embedding.weight)
compare_model_vs_data(model_0, game, dd)



### Model 0
Parameter containing:
tensor([[-0.0504,  0.0093, -0.0226,  0.0165,  0.0072, -0.0277,  0.0177, -0.0237,
          0.0188, -0.0143,  0.0269, -0.0013, -0.0198,  0.0144, -0.0243,  0.0237,
         -0.0140,  0.0181,  0.0318, -0.0157,  0.0148, -0.0034, -0.0324,  0.0208,
         -0.0291, -0.0305,  0.0342, -0.0172,  0.0260, -0.0208, -0.0124,  0.0096],
        [-0.0126,  0.0072,  0.0104, -0.0769, -0.0078, -0.0041, -0.0216,  0.0517,
          0.0385, -0.0091, -0.0166,  0.0048,  0.0219, -0.0037, -0.0307,  0.0049,
         -0.0108,  0.0062,  0.0002, -0.0093, -0.0116, -0.0281, -0.0046, -0.0189,
         -0.0464, -0.0034, -0.0101, -0.0152, -0.0383,  0.0229, -0.0058, -0.0105],
        [-0.0208, -0.0252,  0.0038, -0.0199, -0.0087, -0.0136, -0.0072,  0.0004,
         -0.0083,  0.0022, -0.0049,  0.0290,  0.0091, -0.0186, -0.0067,  0.0133,
         -0.0194, -0.0004,  0.0097,  0.0346, -0.0028, -0.0301,  0.0015,  0.0175,
         -0.0117,  0.0087,  0.0185, -0.0033,  0.0177,  0.0148,  0.0138,

In [19]:
if RUN_GENERATIONS:
    print("\n\n### Model 1")
    print(model_1.action_embedding.weight)
    compare_model_vs_data(model_1, game, dd)



### Model 1
Parameter containing:
tensor([[ 0.1855, -0.1236,  0.0325, -0.2279, -0.1312,  0.2044, -0.0552,  0.0015,
          0.0456, -0.0645, -0.2379,  0.0170,  0.0728,  0.1388, -0.0804,  0.0047,
          0.2121, -0.1895,  0.0768,  0.0613,  0.1133, -0.3043,  0.0052, -0.1532,
          0.2247,  0.2399,  0.0458,  0.2320,  0.2496, -0.0537, -0.2677, -0.1787],
        [ 0.0808, -0.2869,  0.0688, -0.1646,  0.3144,  0.0958,  0.0340, -0.1392,
         -0.2065,  0.4270,  0.2191, -0.1682,  0.0057, -0.0339, -0.0352,  0.3426,
          0.0628,  0.1943, -0.1348, -0.0467, -0.1249, -0.0522,  0.3732,  0.0999,
         -0.0559, -0.1346, -0.1585, -0.2288, -0.0584, -0.0580,  0.0284, -0.2018],
        [ 0.3760, -0.0341, -0.0524, -0.1812, -0.2408,  0.1764, -0.0913, -0.1694,
         -0.2916, -0.1422, -0.1412, -0.0781,  0.1165,  0.0633,  0.1347, -0.4096,
          0.2124, -0.2153, -0.0055, -0.1609, -0.2353,  0.0733, -0.1883, -0.4946,
          0.1866,  0.2968,  0.3776,  0.3554,  0.0164, -0.1493, -0.3089,

## Debug Convergence

Synthetic sanity-check: train on a toy 2-step game where the first action strongly determines the winner. This verifies the value head and training loop can learn simple patterns.


In [20]:
xxx STOP HERE xxx

SyntaxError: invalid syntax (3998300578.py, line 1)

In [ ]:
state_0 = game.initial_state()
all_actions_0 = game.all_actions()

print(all_actions_0)


In [ ]:
import random

def play_random_game_with_fake_reward(game, max_actions) -> dict:
    state = game.initial_state()
    action_history = []
    legal_policies = []
    legal_action_idx_list = []

    all_actions = game.all_actions()
    all_action_idx_map = {action: idx for idx, action in enumerate(all_actions)}

    num_actions = 0
    while not game.is_terminal(state) and num_actions < max_actions:
        current_player = game.current_player_id(state)
        legal_actions = game.legal_actions(state)
        action_idx = random.randrange(len(legal_actions))
        action = legal_actions[action_idx]

        action_history.append(action)
        legal_policies.append(np.ones(len(legal_actions))/len(legal_actions))
        legal_action_idx = np.array([all_action_idx_map[action] for action in legal_actions])
        legal_action_idx_list.append(legal_action_idx)

        state = game.next_state(state, action)
        num_actions += 1

    # Determine outcome
    fake_reward = np.mean(action_history) / len(legal_actions)
    rewards = np.array([fake_reward, 1.0-fake_reward])
    if fake_reward >= 0.5:
        winner = 1
    else:
        winner = 2

    return {
        "winner": winner,
        "rewards": rewards,
        "action_history": action_history,
        "legal_policies": legal_policies,
        "final_state": state,
        "legal_action_idx": legal_action_idx_list,
    }

In [ ]:
play_random_game_with_fake_reward(game, max_actions=2)

In [ ]:
results = [play_random_game_with_fake_reward(game, max_actions=2) for _ in range(100_000)]
print_game_stats(results)


In [ ]:
fake_gen_name = "fake-0"
trajectory_path = write_trajectory_dataset(results, action_vocab, fake_gen_name)


In [ ]:
# fake_model_config = model_config_dict[MODEL_SIZE]
fake_model_config = model_config_dict["large"]
fake_model = create_random_model(fake_model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)

training_splits = [f'gen-{fake_gen_name}']
fake_model, fake_trainer = train_model(fake_model, training_splits, train_config)
save_model(fake_model, fake_trainer, fake_gen_name)

## model_size=tiny
# num decayed parameter tensors: 11, with 1,968 parameters
# num non-decayed parameter tensors: 7, with 50 parameters
# using fused AdamW: False
# step 0: train loss 2.7817, val loss 2.7816
# iter 0/49/488: loss 2.7821, time 2537.56ms
# iter 100/147/488: loss 2.6890, time 53.61ms
# iter 200/245/488: loss 2.6342, time 63.05ms
# iter 300/343/488: loss 2.6187, time 55.31ms
# iter 400/441/488: loss 2.6147, time 61.11ms

## model_size=large
# num decayed parameter tensors: 35, with 1,579,776 parameters
# num non-decayed parameter tensors: 19, with 2,186 parameters
# using fused AdamW: False
# step 0: train loss 2.8087, val loss 2.8088
# iter 0/49/488: loss 2.8099, time 11225.20ms
# iter 100/147/488: loss 2.6065, time 596.91ms
# iter 200/245/488: loss 2.6075, time 618.00ms
# iter 300/343/488: loss 2.6080, time 613.63ms
# iter 400/441/488: loss 2.6051, time 616.39ms

In [ ]:
# for rerun in range(10):
#     print(f"Re-running training for {fake_gen_name} {rerun+1} of 10")
#     fake_model, fake_trainer = train_model(fake_model, training_splits, train_config)
#     save_model(fake_model, fake_trainer, fake_gen_name)

In [ ]:
# [td for td in td_array]
fake_td_array = [TrajectoryDataset(DATA_DIR, split, block_size=n_max_context) for split in training_splits]
fake_unrolled = [(generation+1, d) for generation, td in enumerate(fake_td_array) for d in td]

# gen, d = unrolled[0], 
# d.action[:2]
# d.value[0]

# Inspect training data
fake_dd = defaultdict(lambda: defaultdict(lambda: torch.tensor([0., 0.])))

for gen, d in fake_unrolled:
    for g in ['*', gen]:    
        fake_dd[tuple(tuple(d.action[:0].tolist()))][g] += d.value[0]
        fake_dd[tuple(tuple(d.action[:1].tolist()))][g] += d.value[0]
        fake_dd[tuple(tuple(d.action[:2].tolist()))][g] += d.value[0]
        # fake_dd[tuple(tuple(d.action[:3].tolist()))][g] += d.value[0]

print(f"len(fake_dd) = {len(fake_dd)}")


In [ ]:
fake_model = load_model(fake_gen_name)
compare_model_vs_data(fake_model, game, dd)


In [ ]:
fake_model, fake_trainer = train_model(fake_model, training_splits, train_config)
